# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup as bs

In [2]:
browser = webdriver.Chrome(executable_path='../chromedriver')
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

selectYear = Select(browser.find_element_by_id('ctl05_ddlYear'))
selectYear.select_by_value('2018')
browser.find_element_by_id('ctl05_btnQuery').click()

In [4]:
html_source = browser.page_source
soup = bs(html_source, 'html.parser')
table = soup.find('table', class_='TABLE_G')

keys = []
header = table.find_all('td', class_='no-alt')
for txt in header:
    if (txt.text) != 'ppb' and (txt.text) != 'ppm':
        keys.append(txt.text)
        
    if len(keys) == 2:
        break
keys

['SO2', 'CO']

In [8]:
d = dict()
row = table.find_all('td', attrs={'class': None})

k = ''
v = ''
count = 0
i = 0
key = keys[i]

for content in row:
    if content.text.strip() != '':
        if k == '':
            k = content.text
        elif v == '':
            v = content.text

        if k != '' and v != '':
            try:
                d[key][k] = v
            except:
                d[key] = dict()
                d[key][k] = v
                
            k = ''
            v = ''
            count = count + 1

        if count == 8:
            break
            
d

{'SO2': {'2018/01': '1.80',
  '2018/02': '1.90',
  '2018/03': '2.20',
  '2018/04': '2.30',
  '2018/05': '3.10',
  '2018/06': '2.70',
  '2018/07': '2.20',
  '2018/08': '2.40'}}

### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [20]:
d = dict()
row = table.find_all('td', attrs={'class': None})

k = ''
v = ''
count = 0
i = 0
key = keys[i]

for content in row:
    if content.text.strip() != '':
        
        if k == '':
            k = content.text
        elif v == '':
            v = content.text

        if k != '' and v != '':
            
            if int(k[5:7]) > 8:
                k = ''
                v = ''
                count = 0
                continue
            try:
                d[key][k] = v
            except:
                d[key] = dict()
                d[key][k] = v
                
            k = ''
            v = ''
            count = count + 1
        
        if count == 8:
            count = 0
            
            if key == keys[len(keys)-1]:
                break
            
            i = i + 1
            key = keys[i]
            
d

{'SO2': {'2018/01': '1.80',
  '2018/02': '1.90',
  '2018/03': '2.20',
  '2018/04': '2.30',
  '2018/05': '3.10',
  '2018/06': '2.70',
  '2018/07': '2.20',
  '2018/08': '2.40'},
 'CO': {'2018/01': '0.34',
  '2018/02': '0.44',
  '2018/03': '0.40',
  '2018/04': '0.38',
  '2018/05': '0.34',
  '2018/06': '0.29',
  '2018/07': '0.21',
  '2018/08': '0.30'}}